In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

categoryDF=pd.read_csv("Z:/MSBooks/NLP/FinalProjectFiles/ACDFullData/aspectCategory_new.csv",delimiter='#',encoding = 'utf-8')
del categoryDF['aspectTermPolarity']
categoryDF.head()

,domainID,categoryPolarity,review,aspectCategory
0,app_2,neu,फेसबुक का सिक्योरिटी चैकअप फीचर पॉपअप की तरह य...,gui
1,app_3,neu,"इस पॉपअप बॉक्स में पासवर्ड चेंज करने, लॉगइन अल...",misc
2,app_4,neu,आप यहीं से पुराने लॉगइन सेशंस को हमेशा के लिए ...,gui
3,app_5,neu,ऐसे में कई बार ऐसा होता है कि आपका लॉगइन ओपन ह...,NaN
4,app_6,pos,अब नए फीचर से इस तरह की गलतियां नहीं होंगी और ...,NaN


In [3]:
domain=[]
for id in categoryDF.index:
    domain.append(categoryDF.iloc[id,0][:3])
categoryDF['domain'] = domain
categoryDF.head()

,domainID,categoryPolarity,review,aspectCategory,domain
0,app_2,neu,फेसबुक का सिक्योरिटी चैकअप फीचर पॉपअप की तरह य...,gui,app
1,app_3,neu,"इस पॉपअप बॉक्स में पासवर्ड चेंज करने, लॉगइन अल...",misc,app
2,app_4,neu,आप यहीं से पुराने लॉगइन सेशंस को हमेशा के लिए ...,gui,app
3,app_5,neu,ऐसे में कई बार ऐसा होता है कि आपका लॉगइन ओपन ह...,NaN,app
4,app_6,pos,अब नए फीचर से इस तरह की गलतियां नहीं होंगी और ...,NaN,app


In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(categoryDF, test_size = 0.2)
test.head()

,domainID,categoryPolarity,review,aspectCategory,domain
872,mob_25,neu,एंड्रॉयड 4.4 का ऑपरेटिंग सिस्टम दिया गया है।,software,mob
5182,tra_679,con,हालांकि इन दिनों बावड़ी का एक हिस्सा गिरा हुआ ह...,place,tra
865,mob_18,neg,3जी प्लास्टिक का बना हुआ था जो हाथ से फिसलता था।,design,mob
3237,spe_23,neu,डिज़ाइन काफी ट्रेडिशनल है।,design,spe
993,mob_146,neg,फ्रंट कैमरा के साथ भी थोड़ी-बहुत दिक्कत आई।,hardware,mob


In [4]:
"""categorySizeDF = pd.DataFrame({"count":categoryDF.groupby('domain',sort='False').size()})
categorySizeDF = categorySizeDF.reset_index()
categorySizeDF"""

,domain,count
0,app,229
1,cam,150
2,hea,43
3,hom,84
4,lap,348
5,mob,1141
6,mov,890
7,sma,330
8,spe,47
9,tab,1244


In [5]:
"""temp = categorySizeDF.ix[10]
categorySizeDF.ix[10] = categorySizeDF.ix[11]
categorySizeDF.ix[11]= temp"""

In [6]:
#categorySizeDF

,domain,count
0,app,229
1,cam,150
2,hea,43
3,hom,84
4,lap,348
5,mob,1141
6,mov,890
7,sma,330
8,spe,47
9,tab,1244


In [7]:
"""dfList=[]
currentIndex= 0 
for id in categorySizeDF.index:    
    if id == 0:
        currentIndex=categorySizeDF.iloc[id,1]
        dfList.append(categoryDF[:currentIndex])
        
    else:
        nextIndex= currentIndex  + categorySizeDF.iloc[id,1]
        dfList.append(categoryDF[currentIndex:nextIndex])
        currentIndex=nextIndex"""



In [8]:
"""entertainmentDF = dfList[6].append(dfList[10])
electronicDF = dfList[1]
for i in range(len(dfList)):
    if i>1 and i!=6 and i!=10:
        electronicDF = electronicDF.append(dfList[i])

entertainmentDF = entertainmentDF.dropna(axis=0,how='any')
electronicDF = electronicDF.dropna(axis=0,how='any')"""

In [9]:
#entertainmentDF['review'].to_csv("Z:/MSBooks/NLP/Project/categorysets/eReview.txt",encoding='utf-8', index= False)
#electronicDF['review'].to_csv("Z:/MSBooks/NLP/Project/categorysets/elecReview.txt",encoding='utf-8',index = False)

In [10]:
"""import codecs
import hindiTokenizer
from hindiTokenizer import Tokenizer
import nltk


with codecs.open("Z:/MSBooks/NLP/Project/categorysets/eReview.txt",encoding='utf-8') as eReview:
    eReviewLines = eReview.readlines()

with codecs.open("Z:/MSBooks/NLP/Project/categorysets/elecReview.txt",encoding='utf-8') as elecReview:
    elecReviewLines = elecReview.readlines()"""


In [5]:
import codecs
reviewList=[]
posReview = codecs.open('Z:/MSBooks/NLP/Project/postaggedreviews.txt',encoding='utf-8')
reviewList=[x.strip() for x in posReview.readlines()]

In [6]:
print reviewList[0]

app_2/QC फेसबुक/NN का/PSP सिक्योरिटी/JJ चैकअप/NN फीचर/JJ पॉपअप/NN की/PSP तरह/PSP यूजर्स/NN को/PSP दिखाइ/VM देगा/VAUX ./SYM


In [7]:
from collections import OrderedDict
adjectives=OrderedDict()
for item in reviewList:
    idAndTagged = item.split()
    id = idAndTagged[0]
    taggedSentence = idAndTagged[1:]
    domain = id.split("/")[0]
    definingTerms = [s for s in taggedSentence if "JJ" in s]
    adjTerms = []
    for items in definingTerms:
        word=items.split("/")[0]
        word = word.encode('utf-8')
        adjTerms.append(word)
        adjectives[domain]=adjTerms




In [8]:
adjectiveDF = pd.DataFrame.from_dict(adjectives,orient='index')



adjectiveDF = adjectiveDF.reset_index()
adjectiveDF = adjectiveDF.rename(columns={"index":"reviewID"})
adjectiveDF.head()



,reviewID,0,1,2,3,4,5,6,7
0,app_2,सिक्योरिटी,फीचर,None,None,None,None,None,None
1,app_3,पासवर्ड,मौजूदा,पूरी,बंद,None,None,None,None
2,app_4,पुराने,None,None,None,None,None,None,None
3,app_6,नए,मुश्किल,None,None,None,None,None,None
4,app_9,मुताबिक,कोर,None,None,None,None,None,None


In [10]:
domain=[]
for id in adjectiveDF.index:
    domain.append(adjectiveDF.iloc[id,0][:3])
adjectiveDF['domain'] = domain

adjectiveDF.head()

,reviewID,0,1,2,3,4,5,6,7,domain
0,app_2,सिक्योरिटी,फीचर,None,None,None,None,None,None,app
1,app_3,पासवर्ड,मौजूदा,पूरी,बंद,None,None,None,None,app
2,app_4,पुराने,None,None,None,None,None,None,None,app
3,app_6,नए,मुश्किल,None,None,None,None,None,None,app
4,app_9,मुताबिक,कोर,None,None,None,None,None,None,app


In [15]:
#entAdjectives = entertainmentDF.merge(adjectiveDF,how='inner',left_on='domainID',right_on='reviewID')
#elecAdjectives = electronicDF.merge(adjectiveDF,how='inner',left_on='domainID',right_on='reviewID')

trainAdjectives = train.merge(adjectiveDF,how='inner',left_on='domainID',right_on='reviewID')

In [16]:
#entAdjectives = entAdjectives.drop(['domain_x','reviewID','domain_y'],axis=1)
trainAdjectives = trainAdjectives.drop(['domain_x','reviewID','domain_y'],axis=1)

In [17]:
#elecAdjectives = elecAdjectives.drop(['domain_x','reviewID','domain_y'],axis=1)

In [22]:
"""entLexicon={}
for id in entAdjectives.index:
    category = entAdjectives.iloc[id,3]
    adjectives = entAdjectives.iloc[id][4:]
    adjectives = adjectives.dropna()
    #print adjectives
    for items in adjectives:
        adjList=[]
        if category in entLexicon.keys():
            entLexicon[category].append(items)
        else:
            adjList.append(items)
            entLexicon[category]=adjList
            
elecLexicon={}
for id in elecAdjectives.index:
    category = elecAdjectives.iloc[id,3]
    adjectives = elecAdjectives.iloc[id][4:]
    adjectives = adjectives.dropna()
    for items in adjectives:
        adjList=[]
        if category in elecLexicon.keys():
            elecLexicon[category].append(items)
        else:
            adjList.append(items)
            elecLexicon[category] = adjList"""
trainLexicon={}
for id in trainAdjectives.index:
    category = trainAdjectives.iloc[id,3]
    adjectives = trainAdjectives.iloc[id][4:]
    adjectives = adjectives.dropna()
    for items in adjectives:
        adjList=[]
        if category in trainLexicon.keys():
            trainLexicon[category].append(items)
        else:
            adjList.append(items)
            trainLexicon[category] = adjList
    

In [24]:
#for keys in trainLexicon.keys():
 #   print len(trainLexicon[keys])

In [25]:
"""word = "खाना".decode('utf-8', 'ignore')

import pickle

word2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))
synset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))
synonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))
#synset2Gloss = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetGloss.pk"))
#synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))
#synset2Hyponyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHyponym.pk"))
#synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))


if word2Synset.has_key(word):
    synsets = word2Synset[word]
    for pos in synsets.keys():
        for synset in synsets[pos]:
            if synonyms.has_key(synset):
                print "\t\t\t Synonyms -->", synonyms[synset]"""
                
                
            


'word = "\xe0\xa4\x96\xe0\xa4\xbe\xe0\xa4\xa8\xe0\xa4\xbe".decode(\'utf-8\', \'ignore\')\n\nimport pickle\n\nword2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))\nsynset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))\nsynonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))\n#synset2Gloss = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetGloss.pk"))\n#synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))\n#synset2Hyponyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHyponym.pk"))\n#synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))\n\n\nif word2Synset.has_key(word):\n    synsets = word2Synset[word]\n    for pos in synsets.keys():\n        for synset in synsets[pos]:\n            if synonyms.has_key(synset):\n                print

In [26]:
"""import pickle

word2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))
synset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))
synonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))
synset2Gloss = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetGloss.pk"))
synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))
synset2Hyponyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHyponym.pk"))
synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))"""

In [28]:
import pickle
word2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))
#synset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))
synonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))
def getSynonyms(word):
    """import pickle

    word2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))
    synset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))
    synonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))"""
    #synset2Gloss = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetGloss.pk"))
    #synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))
    #synset2Hyponyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHyponym.pk"))
    #synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))

    synList=[]
    if word2Synset.has_key(word):
        synsets = word2Synset[word]
        for pos in synsets.keys():
            for synset in synsets[pos]:
                if synonyms.has_key(synset):
                    #print "\t\t\t Synonyms -->", synonyms[synset]
                    synDict = synonyms[synset]
                    synList.append(synDict)
    #print len(synList)
    return synList
"""word = "खाना".decode('utf-8', 'ignore')
synList = getSynonyms(word)
for key,value in synList[1].iteritems():
    for item in value:
        print item"""
        
"""for key,item in entLexicon.iteritems():
    for words in item:
        if isinstance(words,str):
            word = words.decode('utf-8','ignore')
            synList = getSynonyms(word)
            if len(synList)>0:
                for k,value in synList[0].iteritems():
                    for syns in value:
                        entLexicon[key].append(syns)
       
        for k,value in synList[1].iteritems():
            for syns in value:
                lexicon[key].append(syns)"""
        
"""for key,item in elecLexicon.iteritems():
    for words in item:
        if isinstance(words,str):
            word = words.decode('utf-8','ignore')
            synList = getSynonyms(word)
            if len(synList)>0:
                for k,value in synList[0].iteritems():
                    for syns in value:
                        elecLexicon[key].append(syns)"""
for key,item in trainLexicon.iteritems():
    for words in item:
        if isinstance(words,str):
            word = words.decode('utf-8','ignore')
            synList = getSynonyms(word)
            if len(synList)>0:
                for k,value in synList[0].iteritems():
                    for syns in value:
                        trainLexicon[key].append(syns)


In [29]:
for keys in trainLexicon.keys():
    print len(trainLexicon[keys])

4608
4
146
34
22
10817
77
11378
12
4935
623
81
370
52
51
231
25
78
53
19
44
20
165
57
2258
1982
337
69
31
633
36
1077
5
92
991
23
186
61
32
713
1
304
47
23
144
320
127
3617
54
1780


In [30]:
import json,ast
with open("Z:/MSBooks/NLP/Project/termextracted/cat_final.txt") as termFile:
    termData = json.load(termFile)
termData = ast.literal_eval(json.dumps(termData, ensure_ascii=False, encoding='utf8'))


In [31]:
"""from collections import OrderedDict
categoryDict=OrderedDict()
for key,items in termData.iteritems():
    for k,v in items.iteritems():
        if len(v)>0:
            for items in v:
                for lexKey,lexItems in entLexicon.iteritems():
                    if items.decode('utf-8','ignore') in lexItems:
                        categoryDict[key] = lexKey
                for lk,li in elecLexicon.iteritems():
                    if items.decode('utf-8','ignore') in li:
                        categoryDict[key] = lk"""
from collections import OrderedDict
categoryDict=OrderedDict()
for key,items in termData.iteritems():
    for k,v in items.iteritems():
        if len(v)>0:
            for items in v:
                for lexKey,lexItems in trainLexicon.iteritems():
                    if items.decode('utf-8','ignore') in lexItems:
                        categoryDict[key] = lexKey

C:\Users\Abijith Sankar\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [33]:
#categoryDict

In [34]:
#predictedCategoryDF = pd.DataFrame.from_dict(categoryDict)
predictedCategoryDF = pd.DataFrame(categoryDict.items(), columns=['reviewID', 'predictedCategory'])
predictedCategoryDF.head()

,reviewID,predictedCategory
0,mov_204,software
1,mob_1393,software
2,tab_641,software
3,tab_765,software
4,tab_760,software


In [46]:
finalDF = test.merge(predictedCategoryDF,left_on="domainID",right_on="reviewID")
finalDF['predictedCategory'].unique()

array([u'misc', u'software', u'hardware', u'software&design&hardware',
       u'scenery', u'place', u'ease_of_use', u'price',
       u'software&hardware', u'performance', u'price&hardware', nan,
       u'design', u'music'], dtype=object)

In [47]:
finalDF = finalDF.dropna()

In [48]:
finalDF.reset_index(drop='True')
len(finalDF['predictedCategory'])

87

In [49]:
from sklearn.metrics import f1_score
f1_score(finalDF['aspectCategory'], finalDF['predictedCategory'], average='micro')  

0.19540229885057475

In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(finalDF['aspectCategory'], finalDF['predictedCategory'])  

0.19540229885057472

In [51]:
for key,value in trainLexicon.iteritems():
    print (key,":")
    for items in value:
        print items
    print ("!!!!!!!!!!!!!!!!!!!!!!")

(nan, ':')
प्राप्त
हास्यास्पद
महसूस
समग्र
छोटा
खूबसूरत
बरबस
सीमित
पसंदीदा
अच्छा
समानांतर
स्थानीय
आदिवासी
मुख्य
अच्छा
साबित
अपूर्व
मैजू
लाल
सफेद
सुंदर
दिलचस्पी
लायक
आनामुडी
परफैक्ट
सवार
रोचक
BBM
नया
BBM
गिनेचुने
प्राचीन
मानवरूपी
प्रतिष्ठित
हबीब
दावत-ए-इश्क
जायकेदार
स्क्रीन
जबरदस्त
खास
पिछली
निम
आयोजित
शीतकालीन
विभिन्न
मुश्किल
मालूम
बरकरार
तेज
विख्यात
छोटे
स्थित
बेहतरीन
तय
वापस
सप्त
द्वारिकापुरी
औसत
स्थित
अंजान
खूबसूरत
शौकीन
साप्ताहिक
नारायणी
अन्य
प्रमुख
पूरे
घने
आदिवासी
सही
तकनीकी
खास
बेहतर
वास्तविक
विश्वव्यापी
खूबसूरत
मजबूत
पूरी
सुरक्षित
बडी
अन्य
तरोताजा
महसूस
प्राचीन
ऐतिहासिक
माफिक
सही
स्वाहिली
बडा
उम्दा
निराशाजनक
मुताबिक
पवित्र
प्रकृतिक
महत्वपूर्ण
एंड्रायड
गहरे
स्लाट
सही
खास
सुंदर
उल्लेखनीय
मुख्य
रूपधारी
पिछला
फुली
फंक्शनल
स्क्रीन
पता
सही
गर्म
गर्म
तैयार
ब्रिटिश
डच
साधारण
बेहतर
दक्षिणी
सीधे
टीविटी
छोटे
क्लासिक
परंपरागत
अच्छा
छोटा
सही
उपयुक्त
असफल
खुला
राष्ट्रीय
सक्षम
घना
आयोजित
प्रदर्शित
स्वाइप
पूरा
तय
आकर्षक
मुताबिक
कोर
कथित
ऑरिजिनल
बेमिसाल
प्रयुक्त
भारतीय
शामिल
रंग-बिरंगे
अद्भूत
स्

In [52]:
with open("Z:/MSBooks/NLP/Project/termextracted/aspect_terms_sentiment_output.txt") as termSentiFile:
    termSentiData = json.load(termSentiFile)
termSentiData = ast.literal_eval(json.dumps(termSentiData, ensure_ascii=False, encoding='utf8'))

In [53]:
termSentiData

{'mov_391': {},
 'mov_393': {},
 'mov_204': {'\xe0\xa4\xad\xe0\xa5\x82\xe0\xa4\xae\xe0\xa4\xbf\xe0\xa4\x95\xe0\xa4\xbe': 1},
 'mov_397': {},
 'mov_398': {},
 'mob_1393': {},
 'mov_53': {},
 'tab_963': {},
 'tab_641': {'\xe0\xa4\xb0\xe0\xa5\x88\xe0\xa4\xae': 1},
 'tab_648': {'\xe0\xa4\xae\xe0\xa5\x87\xe0\xa4\xae\xe0\xa5\x8b\xe0\xa4\xb0\xe0\xa5\x80': 0},
 'lap_437': {},
 'hom_34': {},
 'tab_886': {},
 'hom_37': {},
 'tab_766': {},
 'tab_765': {},
 'hom_39': {},
 'tab_760': {'\xe0\xa4\x95\xe0\xa5\x80\xe0\xa4\xae\xe0\xa4\xa4': 1},
 'app_80': {},
 'tab_110': {},
 'lap_131': {},
 'mob_1421': {},
 'mob_1328': {'\xe0\xa4\xac\xe0\xa4\xa8\xe0\xa4\xbe\xe0\xa4\xb5\xe0\xa4\x9f': -1},
 'tab_118': {'\xe0\xa4\xac\xe0\xa5\x88\xe0\xa4\x9f\xe0\xa4\xb0\xe0\xa5\x80': 0},
 'lap_138': {},
 'app_38': {'\xe0\xa4\xa1\xe0\xa4\xbf\xe0\xa4\xb2\xe0\xa5\x80\xe0\xa4\xb5\xe0\xa4\xb0\xe0\xa5\x80': -1,
  '\xe0\xa4\xae\xe0\xa5\x88\xe0\xa4\xb8\xe0\xa5\x87\xe0\xa4\x9c': 1},
 'tra_474': {},
 'hom_50': {},
 'tel_135': {'\xe0

In [54]:
"""categorySentiScore={}
for key,values in termSentiData.iteritems():
    if len(values)>0:
        for k,v in values.iteritems():
            for entKey,entVal in entLexicon.iteritems():
                if k in entVal:
                    if entKey in categorySentiScore:
                        categorySentiScore[entKey] += v
                    else:
                        categorySentiScore[entKey] = v
            for elecKey,elecVal in elecLexicon.iteritems():
                if k in elecVal:
                    if elecKey in categorySentiScore:
                        categorySentiScore[elecKey] += v
                    else:
                        categorySentiScore[elecKey] = v"""
categorySentiScore={}
for key,values in termSentiData.iteritems():
    if len(values)>0:
        for k,v in values.iteritems():
            for entKey,entVal in trainLexicon.iteritems():
                if k in entVal:
                    if entKey in categorySentiScore:
                        categorySentiScore[entKey] += v
                    else:
                        categorySentiScore[entKey] = v
                    

C:\Users\Abijith Sankar\Anaconda2\lib\site-packages\ipykernel\__main__.py:22: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [55]:
predictedCategory = finalDF['predictedCategory']
predictedCategorySentiment=[]
for category in predictedCategory:
    if category in categorySentiScore.keys():
        if categorySentiScore[category] > 0:
            predictedCategorySentiment.append('pos')
        elif categorySentiScore[category] == 0:
            predictedCategorySentiment.append('neu')
        elif categorySentiScore[category] < 0:
            predictedCategorySentiment.append('neg')
    else:
        predictedCategorySentiment.append('neu')


In [59]:
finalDF['predictedCategorySentiment'] = predictedCategorySentiment
f1_score(finalDF['categoryPolarity'], finalDF['predictedCategorySentiment'], average='micro') 


0.20689655172413793

In [60]:
finalDF

,domainID,categoryPolarity,review,aspectCategory,domain,reviewID,predictedCategory,predictedCategorySentiment
0,mob_1401,neg,हम इस डिवाईस के प्रदर्शन से पूरी तरह संतुष्ट न...,misc,mob,mob_1401,misc,pos
1,mob_908,neu,इसके साथ ही यह नया स्मार्टफ़ोन ड्यूल-सिम सपोर्ट...,hardware,mob,mob_908,software,neg
2,mob_1359,con,"इसके अलावा, आगे ड्रैगनट्रेल ग्लास प्रोटेक्शन द...",misc,mob,mob_1359,misc,pos
3,tab_235,pos,बेंचमार्क टेस्ट में गूगल नेक्सस 7 काफी अच्छा प...,misc,tab,tab_235,software,neg
4,tab_600,pos,"इसके स्पीकर की आवाज़ पहले से ज्यादा तेज़ है , इ...",hardware,tab,tab_600,hardware,pos
5,app_48,neu,यह एप्प अभी सिर्फ एंड्रॉयड डिवाइसेज के लिए ही ...,misc,app,app_48,software,neg
6,tra_108,pos,भारत में लोगों के बीच हिल स्टेशन का क्रेज जबरद...,place,tra,tra_108,software&design&hardware,neu
7,app_32,pos,"आपकी रूचि बनाये रखने के लिए, इस एप्प में काले,...",misc,app,app_32,scenery,neu
8,tra_495,pos,उत्तराखंड की गोद में स्थित धनौल्टी गढ़वाल का म...,place,tra,tra_495,place,neg
10,mov_606,con,"उनका जोर हंसाने पर रहा है, इसलिए कई बार वह जबर...",misc,mov,mov_606,scenery,neu


In [44]:
finalDF.to_csv(path_or_buf="Z:/MSBooks/NLP/Project/termextracted/trialout.csv",index='False',encoding='utf-8',index_label='reviewNum')